# SQD for water on IBM

In [ ]:
from functools import partial

import numpy as np
from pathlib import Path

import pyscf
import pyscf.cc
import pyscf.mcscf

# To get molecular geometries.
import openfermion as of

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator  # For MPS Simulator.

# To run on hardware.
import qiskit_ibm_runtime
from qiskit_ibm_runtime import SamplerV2 as Sampler

import ffsim
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_addon_sqd.fermion import SCIResult, diagonalize_fermionic_hamiltonian, solve_sci_batch

In [ ]:
HARDWARE: bool = True

nshots: int = 5_000

# initial_layout = list(range(80, 85 + 1)) + list(range(90, 95 + 1)) + list(range(100, 105 + 1)) + list(range(110, 115 + 1))  # Miami Jan 1.
# initial_layout = list(range(3, 7 + 1)) + [16, 17] + list(range(23, 27 + 1))  # Boston Jan 4 before calibration.
initial_layout = [131, 132, 133, 119, 134, 135, 138, 139, 150, 151, 152, 153, 154, 155]  # Boston Jan 4 before calibration.
# initial_layout = [94, 105, 106, 107, 108, 109, 112, 113, 124, 125, 126, 127, 128, 132]

layout_pattern: str = "linear"

In [ ]:
ibm_computer: str = "ibm_fez"

if HARDWARE:
    service = qiskit_ibm_runtime.QiskitRuntimeService(name="Q4BIOFLEX")
    computer = service.backend(ibm_computer)
    sampler = Sampler(computer)
else:
    computer = qiskit_ibm_runtime.fake_provider.FakeTorino()
    simulator = AerSimulator(method="matrix_product_state").from_backend(computer)
    sampler = Sampler(simulator)

In [ ]:
path = Path("./hamiltonians/qmd/monomer/sampled_20_15.data")

b2a = 1 / 1.8897259886  # Bohr to Angstrom conversion factor.

atom_lines = []
for line in path.read_text().splitlines():
    if line.startswith("atom"):
        atom_lines.append(line)

geometries = []
for i in range(0, len(atom_lines), 3):
    geom = []
    for line in atom_lines[i:i + 3]:
        parts = line.split()
        x, y, z = map(float, parts[1: 3 + 1])
        element = parts[4]
        geom.append((element, (b2a * x, b2a * y, b2a * z)))
    geometries.append(tuple(geom))

len(geometries)

## Run all

In [ ]:
for geom_index in range(len(geometries):
    print("Geometry index:", geom_index)
    geometry = geometries[geom_index]
    print(geometry)
    
    mol = pyscf.gto.Mole()
    mol.build(
        atom=geometry,
        basis="sto-6g",
    )
    
    n_frozen = 0
    active_space = range(n_frozen, mol.nao_nr())
    
    # Get molecular integrals
    scf = pyscf.scf.RHF(mol).run()
    num_orbitals = len(active_space)
    n_electrons = int(sum(scf.mo_occ[active_space]))
    num_elec_a = (n_electrons + mol.spin) // 2
    num_elec_b = (n_electrons - mol.spin) // 2
    cas = pyscf.mcscf.CASCI(scf, num_orbitals, (num_elec_a, num_elec_b))
    mo = cas.sort_mo(active_space, base=0)
    hcore, nuclear_repulsion_energy = cas.get_h1cas(mo)
    eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), num_orbitals)
    
    # Compute exact energy
    exact_energy = cas.run().e_tot
    
    # Create ansatz.
    ccsd = pyscf.cc.CCSD(scf, frozen=[i for i in range(mol.nao_nr()) if i not in active_space]).run()
    t1 = ccsd.t1
    t2 = ccsd.t2
    
    n_reps = 1
    alpha_alpha_indices = [(p, p + 1) for p in range(num_orbitals - 1)]
    alpha_beta_indices = [(p, p) for p in range(0, num_orbitals, 4)]
    
    ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
        t2=t2,
        t1=t1,
        n_reps=n_reps,
        interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
    )
    
    nelec = (num_elec_a, num_elec_b)
    
    # create an empty quantum circuit
    qubits = QuantumRegister(2 * num_orbitals, name="q")
    circuit = QuantumCircuit(qubits)
    
    # prepare Hartree-Fock state as the reference state and append it to the quantum circuit
    circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), qubits)
    
    # apply the UCJ operator to the reference state
    circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)
    
    # Measure all qubits.
    circuit.measure_all()
    
    # Compile
    if layout_pattern == "staggered":
        spin_a_layout = initial_layout[0::2]
        spin_b_layout = initial_layout[1::2]
    elif layout_pattern == "linear":
        spin_a_layout = initial_layout[: len(initial_layout) // 2]
        spin_b_layout = initial_layout[len(initial_layout) // 2:]
    
    
    initial_layout = spin_a_layout + spin_b_layout
    
    twoq_best = np.inf
    to_run = None
    for trial in range(50):
        pass_manager = generate_preset_pass_manager(
            optimization_level=3, backend=computer, initial_layout=initial_layout, seed_transpiler=trial
        )
        pass_manager.pre_init = ffsim.qiskit.PRE_INIT
        current = pass_manager.run(circuit)
        # print(f"Gate counts (w/ pre-init passes): {current.count_ops()}")
        twoq = current.count_ops().get("cz")
        if twoq < twoq_best:
            twoq_best = twoq
            to_run = current
    
    print(to_run.count_ops())
    
    job = sampler.run([to_run], shots=nshots)
    # job = service.job(job_id="d5bi5nhsmlfc739mh53g")
    print("Job ID:", job.job_id())
    
    bit_array = job.result()[0].data.meas
    counts = job.result()[0].data.meas.get_counts()
    
    
    # SQD options
    energy_tol = 1e-3
    occupancies_tol = 1e-3
    max_iterations = 10
    rng = np.random.default_rng(1)
    
    # Eigenstate solver options
    num_batches = 2
    samples_per_batch = 300
    symmetrize_spin = True
    carryover_threshold = 1e-4
    max_cycle = 200
    
    # Pass options to the built-in eigensolver. If you just want to use the defaults,
    # you can omit this step, in which case you would not specify the sci_solver argument
    # in the call to diagonalize_fermionic_hamiltonian below.
    sci_solver = partial(solve_sci_batch, spin_sq=0.0, max_cycle=max_cycle)
    
    # List to capture intermediate results
    result_history = []
    
    
    def callback(results: list[SCIResult]):
        result_history.append(results)
        iteration = len(result_history)
        # print(f"Iteration {iteration}")
        # for i, result in enumerate(results):
        #     print(f"\tSubsample {i}")
        #     print(f"\t\tEnergy: {result.energy + nuclear_repulsion_energy}")
        #     print(f"\t\tSubspace dimension: {np.prod(result.sci_state.amplitudes.shape)}")
    
    
    result = diagonalize_fermionic_hamiltonian(
        hcore,
        eri,
        bit_array,
        samples_per_batch=samples_per_batch,
        norb=num_orbitals,
        nelec=nelec,
        num_batches=num_batches,
        energy_tol=energy_tol,
        occupancies_tol=occupancies_tol,
        max_iterations=max_iterations,
        sci_solver=sci_solver,
        symmetrize_spin=symmetrize_spin,
        carryover_threshold=carryover_threshold,
        callback=callback,
        seed=rng,
    )
    
    min_e = [
        min(result, key=lambda res: res.energy).energy + nuclear_repulsion_energy
        for result in result_history
    ]
    sqd_energy = min(min_e)
    
    print("FCI:", exact_energy)
    print("SQD:", sqd_energy)
    
    np.savetxt(f"fci_{geom_index}.txt", [exact_energy])
    np.savetxt(f"sqd_{geom_index}.txt", [sqd_energy])